In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
import scipy.sparse as sparse
import traceback
import random
from sklearn import metrics
import re
import implicit
import json
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix, csr_matrix
# from implicit.approximate_als import AnnoyAlternatingLeastSquares,FaissAlternatingLeastSquares,NMSLibAlternatingLeastSquares
# from implicit.nearest_neighbours import BM25Recommender,CosineRecommender,TFIDFRecommender,bm25_weight
# from implicit.bpr import BayesianPersonalizedRanking
# from implicit.lmf import LogisticMatrixFactorization
from implicit.evaluation import train_test_split, ranking_metrics_at_k
print(implicit.__version__)

0.7.2


In [2]:
# with open('hyperparameters.json', 'r') as tc:
#     trainingParams = json.load(tc)
# trainingParams

In [3]:
# factors                 =         trainingParams.get('factors', 100)
# regularization          =         trainingParams.get('regularization',0.01)
# iterations              =         trainingParams.get('iterations',15)
# use_native              =         trainingParams.get('use_native',True)
# use_cg                  =         trainingParams.get('use_cg',True)
# use_gpu                 =         trainingParams.get('use_gpu',False)
# calculate_training_loss =         trainingParams.get('calculate_training_loss',False)
# num_threads             =         trainingParams.get('num_threads',0)
# scenario                =         trainingParams.get('scenario','upsell')
# n_similars              =         trainingParams.get('n_similars',10)
# alpha_val               =         trainingParams.get('alpha_val',40)
# split_ratio             =         trainingParams.get('split_ratio',0.3)
# #return_score            =         trainingParams.get('return_score',False)
# compute_aucscore        =         trainingParams.get('compute_aucscore',False)
# K1                      =         trainingParams.get('K1',100)         
# B                       =         trainingParams.get('B',100)
# learning_rate           =         trainingParams.get('learning_rate',0.01)
# model_name              =         trainingParams.get('model_name','als')
# K                      =         trainingParams.get('K',10)

In [ ]:
#        params = {"factors": factors,"iterations": iterations,"use_native":use_native,"use_cg":use_cg}


factors                 =         40
iterations              =         40
alpha_val               =         40
K                       =         10
use_native              =         True
use_cg                  =         True
use_gpu                 =         False
scenario                =         'upsell'
n_similars              =         5
split_ratio             =         0.3
compute_aucscore        =         True
learning_rate           =         0.01
num_threads             =         0
return_score            =         True

In [5]:
data = pd.read_csv('preprocessed_movie_rec_data.csv')
data

,User,Movie_title,rating,genre
0,1,Toy Story (1995),4.0,"Fantasy, Children, Adventure, Animation, Comedy"
1,5,Toy Story (1995),4.0,"Fantasy, Children, Adventure, Animation, Comedy"
2,7,Toy Story (1995),4.5,"Fantasy, Children, Adventure, Animation, Comedy"
3,15,Toy Story (1995),2.5,"Fantasy, Children, Adventure, Animation, Comedy"
4,17,Toy Story (1995),4.5,"Fantasy, Children, Adventure, Animation, Comedy"
...,...,...,...,...
100831,578,"Young Victoria, The (2009)",4.5,"Drama, Romance"
100832,175,Raise Your Voice (2004),5.0,Romance
100833,578,Cold Creek Manor (2003),2.5,"Drama, Thriller"
100834,578,My Blueberry Nights (2007),4.0,"Drama, Romance"


In [6]:
print(f"number of users:{data.User.nunique()}")
print(f"number of Moves:{data.Movie_title.nunique()}")

number of users:610
number of Moves:9719


In [7]:
####################preprocessing#########################
genre_dict = dict(zip(data.Movie_title, data.genre))



data['User'] = data['User'].astype(str)
data['Movie_title'] = data['Movie_title'].astype(str)
data['Movie_title'] = data['Movie_title'].str.strip()

#get category codes
data['User'] = data['User'].astype("category")
data['Movie_title'] = data['Movie_title'].astype("category")
#creating unique ID's for users and items
data['user_id'] = data['User'].cat.codes
data['item_id'] = data['Movie_title'].cat.codes
        
# creating Mappings and storing into dictionaries............
userID_to_UID = dict(enumerate(data['User'].cat.categories))
UID_to_userID = dict(map(reversed,userID_to_UID.items()))
        
itemID_to_ItemNumber = dict(enumerate(data['Movie_title'].cat.categories))
ItemNumber_to_itemID = dict(map(reversed,itemID_to_ItemNumber.items())) 

        
#creating sparse matrices.....
# The implicit library expects data as a item-user matrix
sparse_item_user = sparse.csr_matrix((data['rating'].astype(float), (data['user_id'], data['item_id'])))

model = AlternatingLeastSquares(factors=factors , iterations=iterations ,use_native=use_native,use_cg=use_cg,random_state=44)

In [8]:
sparse_item_user.get_shape()

(610, 9719)

In [9]:
sparse_item_user

<610x9719 sparse matrix of type '<class 'numpy.float64'>'
	with 100832 stored elements in Compressed Sparse Row format>

In [10]:
if compute_aucscore:
    # splitting data into train and test....    
    train, test = train_test_split(sparse_item_user, train_percentage = split_ratio)       
    print('Training implicit model')
    #ALS Model.....
    model.fit(sparse_item_user * alpha_val)
    item_vecs = model.item_factors
    user_vecs = model.user_factors
    metrics = ranking_metrics_at_k(model, train, test, K=K,show_progress=True, num_threads=num_threads)
    print('Training complete.')
    scores = [str(key)+'_'+str(value) for key,value in metrics.items()]
    all_metrics = '-'.join(x for x in scores)   
else:
    model.fit(sparse_item_user.T * alpha_val)
    all_metrics = None
    print('Training complete.')


Training implicit model


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/610 [00:00<?, ?it/s]

Training complete.


In [11]:
all_metrics

'precision_0.21256355584713793-map_0.12012692836697524-ndcg_0.1905005352313823-auc_0.5441508311319115'

In [12]:
item_vecs = model.item_factors
user_vecs = model.user_factors
print(item_vecs.shape,user_vecs.shape)

(9719, 40) (610, 40)


In [14]:
user = 1
item = 'Little Ashes (2008)'

In [15]:
explaination = model.explain(int(userID_to_UID[user]), sparse_item_user, ItemNumber_to_itemID[item], user_weights=None, N=5)
explaination

(0.0656232494065911,
 [(5173, 0.010445630117055744),
  (8722, 0.007933309530247908),
  (1377, 0.006502952837965175),
  (2718, 0.006380700632856531),
  (4972, 0.006286520420704237)],
 (array([[ 0.68077951, -0.02593145,  0.08463157, ...,  0.41847501,
           0.18699379,  0.04034979],
         [-0.0176536 ,  0.64260207,  0.08039009, ..., -0.11207547,
          -0.01677036, -0.01313411],
         [ 0.05761544,  0.04946422,  0.56519704, ...,  0.04810501,
           0.03284809,  0.04586736],
         ...,
         [ 0.28488921, -0.08287159,  0.05359525, ...,  0.54589694,
           0.0546747 ,  0.02649079],
         [ 0.12730154, -0.01562569,  0.03304305, ...,  0.32686752,
           0.53475439,  0.00446471],
         [ 0.02746931, -0.00948633,  0.02828311, ...,  0.07570201,
           0.02666   ,  0.48442618]]),
  False))

In [16]:
def get_similar_items(itemid,n_similars):
    similar_items = []
    ids,scores = model.similar_items(ItemNumber_to_itemID[itemid],n_similars,filter_items = [ItemNumber_to_itemID[itemid]])
    for x,y in zip(ids,scores):
        similar_items.append(itemID_to_ItemNumber[x])
    return similar_items

def get_recommendations(userid,n_similars,sparse_item_user):# sparse_item_user
    recommended_items = []
    user_interactions = sparse_item_user.tocsr()
    ids,scores = model.recommend(int(userID_to_UID[userid]),user_interactions.getrow(userid), N = n_similars,filter_already_liked_items=True)#sparse_item_user[userID_to_UID[userid]]
    for x,y in zip(ids,scores):
        recommended_items.append(itemID_to_ItemNumber[x])
    return recommended_items

In [17]:
get_similar_items(item,5)

['Cold Creek Manor (2003)',
 'My Blueberry Nights (2007)',
 'Cheaper by the Dozen (1950)',
 'Letters to Juliet (2010)',
 'Young Victoria, The (2009)']

In [18]:
userID_to_UID[user]
#sparse_item_user[userID_to_UID[user]]
print(type(userID_to_UID[user]))

<class 'str'>


In [19]:
get_recommendations(user,5,sparse_item_user)

['Tin Cup (1996)',
 "What's Eating Gilbert Grape (1993)",
 'Player, The (1992)',
 'Bonnie and Clyde (1967)',
 'Toy Story 2 (1999)']

In [24]:
users = model.similar_users(int(userID_to_UID[user]),5,filter_users = [int(userID_to_UID[user])])
users

(array([368, 450, 332, 177, 460]),
 array([0.6156651 , 0.56023926, 0.55224854, 0.5522269 , 0.5352689 ],
       dtype=float32))

In [ ]:
# ['Say Anything... (1989)',
#  'Dancer in the Dark (2000)',
#  'Lost in Translation (2003)',
#  "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
#  'Toy Story 2 (1999)']

In [104]:
UID_to_userID[user], IId_to_ItemID

KeyError: 1

In [105]:
ItemNumber_to_itemID[item], itemID_to_ItemNumber[5108]

(5108, 'Little Ashes (2008)')

In [106]:
ids,scores = model.similar_items(5108,5)
print(itemID_to_ItemNumber[5108])
print(ids,scores)

Little Ashes (2008)
[5108 1877 5928 1700 9662] [1.         0.99999225 0.9999663  0.92081267 0.8451694 ]


In [107]:
for x,y in zip(ids,scores):
    print(itemID_to_ItemNumber[x],y)

Little Ashes (2008) 1.0
Cold Creek Manor (2003) 0.99999225
My Blueberry Nights (2007) 0.9999663
Cheaper by the Dozen (1950) 0.92081267
Young Victoria, The (2009) 0.8451694


In [108]:
temp_list = [item_tuple for item_tuple in items if 5!=item_tuple[0]]

temp_list

NameError: name 'items' is not defined

In [109]:
up_cell_list = []
for item in temp_list:
    print(item)
    idx, score = item
    if return_score == True:
        up_cell_list.append(str(itemID_to_ItemNumber[idx]) + "," + str(score))
    if return_score == False:
        up_cell_list.append(str(itemID_to_ItemNumber[idx]))

NameError: name 'temp_list' is not defined

In [13]:
#c_list = cross_sell(user,sparse_item_user,n_similars,return_score)
u_list = up_sell(item,n_similars,return_score)

IndexError: index 5108 is out of bounds for axis 0 with size 610

In [ ]:
print(c_list,u_list)